In [3]:
import json
import pandas as pd
from pathlib import Path

# --- Configuración de Rutas ---
JSONL_FILE_PATH = "data/json/batch_vwx6bfxuDSrlSctyJrMgionx_output.jsonl"
CSV_INPUT_PATH = "data/clean/delitos_imagenes_santiago_limpio.csv"
CSV_OUTPUT_PATH =  "data/inferences/resultados_finales_santiago.csv"

def read_jsonl(file_path: str) -> list:
    """Lee un archivo .jsonl de forma segura y lo devuelve como una lista de diccionarios."""
    path_obj = Path(file_path)
    if not path_obj.exists():
        print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
        return []
    
    with open(path_obj, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

def process_batch_results(json_lines: list) -> pd.DataFrame:
    """
    Procesa la lista de resultados del batch y la convierte en un DataFrame limpio.
    Extrae el ID, la descripción de la IA y el resultado JSON completo.
    """
    processed_data = []
    for res in json_lines:
        # Usar .get() para evitar errores si una clave no existe
        custom_id = res.get('custom_id')
        
        # Omitir si no hay ID o si la API reportó un error para esta solicitud
        if not custom_id or res.get('error'):
            print(f"⚠️ Omitiendo registro sin 'custom_id' o con error: {res.get('id')}")
            continue

        # Navegar de forma segura por la estructura anidada del JSON
        response_body = res.get('response', {}).get('body', {})
        choices = response_body.get('choices', [{}])
        message = choices[0].get('message', {})
        description = message.get('content', '') # Este es el texto generado por la IA

        processed_data.append({
            'nombre_foto': custom_id,        # La clave para unir
            'descripcion_ia': description,   # El resultado útil y limpio
            'raw_result_json': json.dumps(res) # La línea JSON completa, como querías
        })
        
    return pd.DataFrame(processed_data)

def main():
    """Función principal que orquesta todo el proceso."""
    print("🚀 Iniciando el procesamiento de resultados del batch de OpenAI.")

    # 1. Leer y procesar los resultados del archivo .jsonl
    json_lines = read_jsonl(JSONL_FILE_PATH)
    if not json_lines:
        print("Proceso finalizado. No hay datos que procesar.")
        return
        
    results_df = process_batch_results(json_lines)
    print(f"✅ Se procesaron {len(results_df)} resultados válidos del archivo JSONL.")

    # 2. Leer el archivo CSV original con los datos de entrada
    if not Path(CSV_INPUT_PATH).exists():
        print(f"❌ Error: No se encontró el archivo CSV de entrada: {CSV_INPUT_PATH}")
        return
    df_original = pd.read_csv(CSV_INPUT_PATH)
    print(f"✅ Se cargaron {len(df_original)} registros del CSV original.")

    # 3. Forzar consistencia de tipos de datos en la columna de unión (¡Paso Crítico!)
    print("🔧 Homologando tipos de dato en la columna 'nombre_foto' para un merge seguro...")
    df_original['nombre_foto'] = df_original['nombre_foto'].astype(str)
    results_df['nombre_foto'] = results_df['nombre_foto'].astype(str)

    # 4. Unir ambos DataFrames usando la clave 'nombre_foto'
    print("🔗 Uniendo datos originales con los resultados de la IA...")
    df_final = pd.merge(df_original, results_df, on='nombre_foto', how='left')

    # 5. Guardar el resultado final
    df_final.to_csv(CSV_OUTPUT_PATH, index=False, encoding='utf-8')
    print(f"\n🎉 ¡Éxito! Archivo final guardado en: {CSV_OUTPUT_PATH}")
    print(f"El archivo contiene {len(df_final)} filas y {len(df_final.columns)} columnas.")

if __name__ == "__main__":
    main()

🚀 Iniciando el procesamiento de resultados del batch de OpenAI.
✅ Se procesaron 4817 resultados válidos del archivo JSONL.
✅ Se cargaron 4817 registros del CSV original.
🔧 Homologando tipos de dato en la columna 'nombre_foto' para un merge seguro...
🔗 Uniendo datos originales con los resultados de la IA...

🎉 ¡Éxito! Archivo final guardado en: data/inferences/resultados_finales_santiago.csv
El archivo contiene 4817 filas y 27 columnas.


In [15]:
jsonl_file_path = "data/json/batch_vwx6bfxuDSrlSctyJrMgionx_output.jsonl" # Resultados en json
csv_file_path= "data/inferences/batch_results_3_chunk_15_09_2024.csv"





def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        # Leer todas las líneas del archivo
        lines = file.readlines()
        
        # Convertir cada línea en un diccionario Python
        json_lines = [json.loads(line) for line in lines]
        
    return json_lines
def process_json_data(json_lines):
    # Crear un DataFrame a partir de la lista de diccionarios
    df = pd.json_normalize(json_lines)
    
    return df
# Convertir a CSV
def convert_to_csv(json_lines, csv_path):
    df = process_json_data(json_lines)
    df.to_csv(csv_path, index=False)

json_lines = read_jsonl(jsonl_file_path)
convert_to_csv(json_lines, csv_file_path)

print(f"Archivo CSV guardado en: {csv_file_path}")

batch_df = pd.read_csv(csv_file_path)
import pandas as pd

results_list = []

json_dict = {res['custom_id']: res for res in json_lines}

for res in json_lines:
    task_id = res['custom_id']
    # Obtener el índice del task_id
    index = int(task_id.split('-')[-1])
    result = res['response']['body']['choices'][0]['message']['content']
    
    # Almacenar el resultado, el índice y la línea JSON completa en la lista
    results_list.append({
        'index': index,
        'result': result,
        'output_batch': json_dict.get(task_id)  # Agregar la línea JSON completa
    })

results_df = pd.DataFrame(results_list)
df = pd.read_csv('data/clean/delitos_imagenes_santiago_limpio.csv') # csv de delitos
df_results = df.merge(results_df, left_index=True, right_on='index', how='left')

df_batch_merge_input = df_results.copy()

# df_final
columnas_a_agregar = ['nombre_foto', 'result'] 

df_merged = df.merge(df_batch_merge_input[columnas_a_agregar], on='nombre_foto', how='left')



# df_final
columnas_a_agregar = ['nombre_foto', 'result'] 

df_merged = df.merge(df_batch_merge_input[columnas_a_agregar], on='nombre_foto', how='left')

df_merged
df_merged.to_csv('data/inferences/delitos_imagenes_santiago_limpio_con_descripcion.csv', index=False)



Archivo CSV guardado en: data/inferences/batch_results_3_chunk_15_09_2024.csv
